In [1]:
import pandas as pd
import pymrmr
from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneOut
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import matthews_corrcoef
import re

### Data Preparation

In [2]:
# read Cell classes
series_txt = 'GSE84465_series_matrix.txt'
with open(series_txt, "r") as series:
    for line in series:
        if re.match("^!Sample_characteristics_ch1\s+\"tissue:", line):
            cell_class = line.rstrip()
            cell_class = re.split('"\t"tissue: |!Sample_characteristics_ch1\t"tissue: |"', cell_class)[1:-1]
        elif re.match("^!Sample_description\t\"\d+", line):
            cell_id = line.rstrip()
            cell_id = re.split('"\t"|!Sample_description\t"|"', cell_id)[1:-1]
series.close()

# Create a Y dataframe
id_class = list(zip(cell_id, cell_class))
Y = pd.DataFrame(id_class, columns=['Cell ID','Cell class'])
Y

,Cell ID,Cell class
0,1001000173.G8,Tumor
1,1001000173.D4,Tumor
2,1001000173.B4,Tumor
3,1001000173.A2,Tumor
4,1001000173.E2,Tumor
...,...,...
3584,1001000271.F1,Tumor
3585,1001000271.D4,Tumor
3586,1001000271.C1,Tumor
3587,1001000271.H7,Tumor


In [3]:
df_complete = pd.read_csv('GSE84465_GBM_All_data.csv', delimiter=' ')
df_complete

,1001000173.G8,1001000173.D4,1001000173.B4,1001000173.A2,1001000173.E2,1001000173.F6,1001000173.B12,1001000173.D6,1001000173.H1,1001000173.E4,...,1001000271.C2,1001000271.D1,1001000271.B1,1001000271.A3,1001000271.E1,1001000271.F1,1001000271.D4,1001000271.C1,1001000271.H7,1001000271.H2
1/2-SBSRNA4,1,0,0,0,0,0,0,64,0,0,...,0,0,0,0,0,0,0,0,0,0
A1BG,0,0,0,0,0,0,53,0,0,229,...,7,2,0,0,0,0,61,0,0,0
A1BG-AS1,0,0,0,1,0,0,24,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1CF,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2LD1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
no_feature,429554,73969,185910,355266,536444,476308,185700,373777,344667,273108,...,331208,48936,290280,476066,150601,223514,173101,349266,3117,391230
ambiguous,8372,2297,12481,11386,8969,6008,14784,11346,8228,13076,...,5791,1022,4994,7266,6384,3400,3605,8243,34,6004
too_low_aQual,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
not_aligned,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# make samples as rows
df_complete = df_complete.T
df_complete

,1/2-SBSRNA4,A1BG,A1BG-AS1,A1CF,A2LD1,A2M,A2ML1,A2MP1,A4GALT,A4GNT,...,ZYG11B,ZYX,ZZEF1,ZZZ3,tAKR,no_feature,ambiguous,too_low_aQual,not_aligned,alignment_not_unique
1001000173.G8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,429554,8372,0,0,111288
1001000173.D4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,73969,2297,0,0,19969
1001000173.B4,0,0,0,0,0,0,0,0,0,0,...,0,834,0,0,0,185910,12481,0,0,132812
1001000173.A2,0,0,1,0,0,0,34,0,0,0,...,0,0,33,1,0,355266,11386,0,0,131212
1001000173.E2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,536444,8969,0,0,145595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001000271.F1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,223514,3400,0,0,108873
1001000271.D4,0,61,0,0,0,2,0,0,0,0,...,0,0,0,0,0,173101,3605,0,0,64761
1001000271.C1,0,0,0,0,0,61,0,0,2,0,...,0,18,0,64,0,349266,8243,0,0,114882
1001000271.H7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3117,34,0,0,507


In [5]:
# append "Labels" column
df_complete["Labels"] = cell_class
df_complete

,1/2-SBSRNA4,A1BG,A1BG-AS1,A1CF,A2LD1,A2M,A2ML1,A2MP1,A4GALT,A4GNT,...,ZYX,ZZEF1,ZZZ3,tAKR,no_feature,ambiguous,too_low_aQual,not_aligned,alignment_not_unique,Labels
1001000173.G8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,429554,8372,0,0,111288,Tumor
1001000173.D4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,73969,2297,0,0,19969,Tumor
1001000173.B4,0,0,0,0,0,0,0,0,0,0,...,834,0,0,0,185910,12481,0,0,132812,Tumor
1001000173.A2,0,0,1,0,0,0,34,0,0,0,...,0,33,1,0,355266,11386,0,0,131212,Tumor
1001000173.E2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,536444,8969,0,0,145595,Tumor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001000271.F1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,223514,3400,0,0,108873,Tumor
1001000271.D4,0,61,0,0,0,2,0,0,0,0,...,0,0,0,0,173101,3605,0,0,64761,Tumor
1001000271.C1,0,0,0,0,0,61,0,0,2,0,...,18,0,64,0,349266,8243,0,0,114882,Tumor
1001000271.H7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3117,34,0,0,507,Tumor


In [6]:
# remove the last 6 columns 
df = df_complete.iloc[: , :-6]
df

,1/2-SBSRNA4,A1BG,A1BG-AS1,A1CF,A2LD1,A2M,A2ML1,A2MP1,A4GALT,A4GNT,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,tAKR
1001000173.G8,1,0,0,0,0,0,0,0,0,0,...,0,30,0,0,0,0,0,0,0,0
1001000173.D4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1001000173.B4,0,0,0,0,0,0,0,0,0,0,...,0,0,40,33,0,0,834,0,0,0
1001000173.A2,0,0,1,0,0,0,34,0,0,0,...,0,0,41,61,0,0,0,33,1,0
1001000173.E2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,439,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001000271.F1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1001000271.D4,0,61,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1001000271.C1,0,0,0,0,0,61,0,0,2,0,...,0,1,98,19,0,0,18,0,64,0
1001000271.H7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# remove columns and rows that have sum = 0 -> no information/expression
df = df.loc[(df.sum(axis=1) != 0), (df.sum(axis=0) != 0)]

In [9]:
# Data normilizations
scaler = StandardScaler()
scaler.fit(df)
scaled = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled, columns=df.columns)
scaled_df

,1/2-SBSRNA4,A1BG,A1BG-AS1,A1CF,A2LD1,A2M,A2ML1,A2MP1,A4GALT,A4GNT,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,tAKR
0,-0.029619,-0.240253,-0.094747,-0.026742,-0.09132,-0.482904,-0.095393,-0.049328,-0.038438,-0.035642,...,-0.175459,1.424694,-0.166165,-0.320590,-0.044885,-0.197845,-0.519129,-0.233546,-0.212233,-0.038795
1,-0.070584,-0.240253,-0.094747,-0.026742,-0.09132,-0.482904,-0.095393,-0.049328,-0.038438,-0.035642,...,-0.175459,-0.103026,-0.166165,-0.320590,-0.044885,-0.197845,-0.519129,-0.233546,-0.212233,-0.038795
2,-0.070584,-0.240253,-0.094747,-0.026742,-0.09132,-0.482904,-0.095393,-0.049328,-0.038438,-0.035642,...,-0.175459,-0.103026,0.580251,0.006859,-0.044885,-0.197845,4.037062,-0.233546,-0.212233,-0.038795
3,-0.070584,-0.240253,0.068737,-0.026742,-0.09132,-0.482904,1.517742,-0.049328,-0.038438,-0.035642,...,-0.175459,-0.103026,0.598911,0.284695,-0.044885,-0.197845,-0.519129,0.120454,-0.198902,-0.038795
4,-0.070584,-0.240253,-0.094747,-0.026742,-0.09132,-0.482904,-0.095393,-0.049328,-0.038438,-0.035642,...,-0.175459,-0.103026,-0.166165,4.035477,-0.044885,-0.197845,-0.519129,-0.222819,-0.212233,-0.038795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3584,-0.070584,-0.240253,-0.094747,-0.026742,-0.09132,-0.482904,-0.095393,-0.049328,-0.038438,-0.035642,...,-0.175459,-0.103026,-0.166165,-0.320590,-0.044885,-0.197845,-0.519129,-0.233546,-0.212233,-0.038795
3585,-0.070584,2.219683,-0.094747,-0.026742,-0.09132,-0.481361,-0.095393,-0.049328,-0.038438,-0.035642,...,-0.175459,-0.103026,-0.166165,-0.320590,-0.044885,-0.197845,-0.519129,-0.233546,-0.212233,-0.038795
3586,-0.070584,-0.240253,-0.094747,-0.026742,-0.09132,-0.435850,-0.095393,-0.049328,0.288078,-0.035642,...,-0.175459,-0.052102,1.662554,-0.132059,-0.044885,-0.197845,-0.420794,-0.233546,0.640947,-0.038795
3587,-0.070584,-0.240253,-0.094747,-0.026742,-0.09132,-0.482904,-0.095393,-0.049328,-0.038438,-0.035642,...,-0.175459,-0.103026,-0.166165,-0.320590,-0.044885,-0.197845,-0.519129,-0.233546,-0.212233,-0.038795


### mRMR Feature Selection

In [25]:
# MIQ -> Mutual Information Quotient
# MID -> Mutual Information Difference
mrmr_features = pymrmr.mRMR(scaled_df, 'MID', 100) # dataframe, feature selection scheme, # of features



 *** This program and the respective minimum Redundancy Maximum Relevance (mRMR) 
     algorithm were developed by Hanchuan Peng <hanchuan.peng@gmail.com>for
     the paper 
     "Feature selection based on mutual information: criteria of 
      max-dependency, max-relevance, and min-redundancy,"
      Hanchuan Peng, Fuhui Long, and Chris Ding, 
      IEEE Transactions on Pattern Analysis and Machine Intelligence,
      Vol. 27, No. 8, pp.1226-1238, 2005.


*** MaxRel features ***
Order 	 Fea 	 Name 	 Score
1 	 5512 	 ERCC-00002 	 0.258
2 	 5563 	 ERCC-00096 	 0.255
3 	 5550 	 ERCC-00074 	 0.253
4 	 5578 	 ERCC-00130 	 0.253
5 	 207 	 ACTB 	 0.244
6 	 5568 	 ERCC-00108 	 0.243
7 	 5572 	 ERCC-00113 	 0.242
8 	 5514 	 ERCC-00004 	 0.242
9 	 11339 	 MALAT1 	 0.240
10 	 5581 	 ERCC-00136 	 0.237
11 	 5602 	 ERCC-00171 	 0.236
12 	 5536 	 ERCC-00046 	 0.236
13 	 5515 	 ERCC-00009 	 0.232
14 	 6693 	 FTH1 	 0.232
15 	 6846 	 GAPDH 	 0.231
16 	 6696 	 FTL 	 0.226
17 	 19983 	 TMSB4X 	 0.22

MemoryError: std::bad_array_new_length

In [17]:
# # read text file into pandas DataFrame
# mrmr_features = pd.read_csv("features.txt", sep="\t")
  
# # display DataFrame
# mrmr_features

,Order,Fea,Name,Score
0,1,5512,ERCC-00002,0.258
1,2,5563,ERCC-00096,0.255
2,3,5550,ERCC-00074,0.253
3,4,5578,ERCC-00130,0.253
4,5,207,ACTB,0.244
...,...,...,...,...
95,96,7384,GPX1,0.168
96,97,5077,DUSP1,0.168
97,98,7895,HLA-B,0.168
98,99,17062,SAT1,0.167


### SVM Construction

In [ ]:
svms = {}
for i in range(1, 101):
  svms[i] = make_pipeline(StandardScaler(), SVC(gamma='auto'))
  # X is increased based on the mrmr features
  X = df
  y = Y
  svms[i].fit(X, y)

### SVM Evaluation

In [ ]:
loo = LeaveOneOut()
loo.get_n_splits(mrmr_array)

In [ ]:
math_cor_coef = {}
for index, svm in enumerate(svms):
  # predict gene based on svm
  y_pred = svm.predict(gene)

  # get the true value
  y_true = 0

  math_cor_coef[index] = matthews_corrcoef(y_true, y_pred)


